In [ ]:
import os

os.makedirs("articles/review", exist_ok=True)

from script.cropPage import *
from script.textMining import *
from script.textSegments import *

In [ ]:
import requests

url = "https://vuir.vu.edu.au/43558/1/Ep43558.pdf"

# Extract filename and append '.pdf'
filename = url.split("/")[-1]

# Full path where the file will be saved
path = os.path.join("articles", filename)

# Send a GET request to the URL with headers
response = requests.get(url, stream=True)

# Save the file if the request was successful
assert response.status_code == 200, response.status_code

with open(path, "wb") as f:
    for chunk in response.iter_content(chunk_size=8192):
        f.write(chunk)

# 1) Crop page


In [ ]:
# Directory containing PDFs to process
pdfsDirectory = "articles"
totalFiles = len(os.listdir(pdfsDirectory))

# Output directory for cleaned text files
outputDirectory = os.path.join(pdfsDirectory, "output")

# Crop all PDFs in the directory
cropAllPdfs(pdfsDirectory, outputDirectory, totalFiles)

# Test single PDF
# pdfPath= "review/1-s2.0-S0143416016300094-main.pdf"
# cropPDFMargins(pdfPath, outputDirectory)

# 2) Extract text

Table not working yet


In [ ]:
# Directory containing your text files
text_files_directory = "articles/output/textOutput"

# Directory containing PDFs to process
pdfs_directory = "articles/output"

# Output directory for cleaned text files
output_directory = os.path.join(pdfs_directory, "textOutput")

# Process all PDFs in the directory
process_all_pdfs(pdfs_directory, output_directory)
process_text_files(text_files_directory)

# 3) Text segment

Get embeddings


In [ ]:
# List of PDF file names
txt_directory = "articles/output/textOutput"
output_text_directory = "text_output"
# Configure OpenAI API key
key_file_path = "api_key"

if "OPENAI_API_KEY" in os.environ:
    with open(key_file_path, "w") as f:
        f.write(os.environ["OPENAI_API_KEY"])

process_files(txt_directory, output_text_directory, key_file_path)

# 4) Query text
